In [1]:
import os
from random import shuffle
import gensim
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import re
from os.path import join
from gensim.models.doc2vec import Doc2Vec
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from gensim.utils import simple_preprocess
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords

import logging
import multiprocessing
import tensorflow as tf
# import tensorflow.keras as keras


from csv import reader
import pandas as pd
from io import StringIO

from nltk.corpus import reuters
import nltk
nltk.download('reuters')
nltk.download('punkt')

from sklearn.preprocessing import MultiLabelBinarizer

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\eogha\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eogha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
class D2V:

    def __init__(self):
        cores = multiprocessing.cpu_count()
        self.__model = Doc2Vec(dm=0,
                               vector_size=300,
                               min_count=2,
                               epochs=70,
                               workers=cores - 1)

    
    def train(self, train_corpus):
        self.__model.build_vocab(train_corpus)
        self.__model.train(train_corpus, total_examples=self.__model.corpus_count, epochs=self.__model.epochs)
        return 1

    def save(self, folder_path, filename):
        self.__model.save(join(folder_path, filename))

    def load(self, folder_path, filename):
        self.__model = Doc2Vec.load(join(folder_path, filename))

    def infer_doc(self, doc):
        return self.__model.infer_vector(doc)

    def get_vector(self, id):
        return self.__model.docvecs[id]

    def get_similar(self, doc):
        return self.__model.docvecs.most_similar([doc])

    def get_doc_vec(self, identifier: str):
        return self.__model.docvecs[identifier]

In [4]:
model_name = "Reuters_test.d2v"

In [5]:
# doc2vec_model_location = "G:/FYP_Work/doc2vec_models"
doc2vec_model_location = "C:/Users/eogha/Documents/Workspace/doc2vec_models"

In [6]:
taggedDocuments = [TaggedDocument(words=gensim.utils.simple_preprocess(remove_stopwords(reuters.raw(fileId).lower())), tags=[i]) for i, fileId in enumerate(reuters.fileids())]

In [7]:
print(taggedDocuments[2])

TaggedDocument(['japan', 'revise', 'long', 'term', 'energy', 'demand', 'downwards', 'ministry', 'international', 'trade', 'industry', 'miti', 'revise', 'long', 'term', 'energy', 'supply', 'demand', 'outlook', 'august', 'meet', 'forecast', 'downtrend', 'japanese', 'energy', 'demand', 'ministry', 'officials', 'said', 'miti', 'expected', 'lower', 'projection', 'primary', 'energy', 'supplies', 'year', 'mln', 'kilolitres', 'kl', 'mln', 'said', 'decision', 'follows', 'emergence', 'structural', 'changes', 'japanese', 'industry', 'following', 'rise', 'value', 'yen', 'decline', 'domestic', 'electric', 'power', 'demand', 'miti', 'planning', 'work', 'revised', 'energy', 'supply', 'demand', 'outlook', 'deliberations', 'committee', 'meetings', 'agency', 'natural', 'resources', 'energy', 'officials', 'said', 'said', 'miti', 'review', 'breakdown', 'energy', 'supply', 'sources', 'including', 'oil', 'nuclear', 'coal', 'natural', 'gas', 'nuclear', 'energy', 'provided', 'bulk', 'japan', 'electric', 'powe

In [8]:
d2v = D2V()

In [9]:
result = d2v.train(taggedDocuments)
if result == 1:
        d2v.save(doc2vec_model_location, model_name)
        print("Training Complete!")

Training Complete!


In [10]:
train_articles = [{'raw': reuters.raw(fileId), 'categories': reuters.categories(fileId)} for fileId in reuters.fileids() if fileId.startswith('training/')]
test_articles = [{'raw': reuters.raw(fileId), 'categories': reuters.categories(fileId)} for fileId in reuters.fileids() if fileId.startswith('test/')]

In [11]:
d2v.load(doc2vec_model_location, model_name)

In [12]:
labelBinarizer = MultiLabelBinarizer()
labelBinarizer.fit([reuters.categories(fileId) for fileId in reuters.fileids()])

MultiLabelBinarizer()

In [13]:
train_data = [d2v.infer_doc(gensim.utils.simple_preprocess(remove_stopwords(article['raw'].lower()))) for article in train_articles]
print("train_data Complete")
test_data = [d2v.infer_doc(gensim.utils.simple_preprocess(remove_stopwords(article['raw'].lower()))) for article in test_articles]
print("test_data Complete")
train_labels = labelBinarizer.transform([article['categories'] for article in train_articles])
print("train_labels Complete")
test_labels = labelBinarizer.transform([article['categories'] for article in test_articles])
print("test_labels Complete")
train_data, test_data, train_labels, test_labels = np.asarray(train_data), np.asarray(test_data), np.asarray(train_labels), np.asarray(test_labels)

train_data Complete
test_data Complete
train_labels Complete
test_labels Complete


In [14]:
doc2vec_dimensions = 300

In [15]:
train_data = np.reshape(train_data, (len(train_data),300,1))
print(train_data.shape)

(7769, 300, 1)


In [16]:
train_labels = np.reshape(train_labels, (len(train_labels),90,1))
print(train_labels.shape)

(7769, 90, 1)


In [17]:
test_data = np.reshape(test_data, (len(test_data),300,1))
test_labels = np.reshape(test_labels, (len(test_labels),90,1))

In [18]:
test_data.shape

(3019, 300, 1)

In [19]:
train_labels = tf.squeeze(train_labels, axis=-1)
test_labels = tf.squeeze(test_labels, axis=-1)

In [20]:
from tensorflow.keras.layers import Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [21]:
model_training = Sequential()
# input_layer = keras.Input(shape=(300,1))
model_training.add(InputLayer(input_shape=(300,1)))
model_training.add(Conv1D(filters=32, kernel_size=8, padding='same', activation='relu'))
model_training.add(Dropout(0.5))
model_training.add(Dense(512))
model_training.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model_training.add(Dropout(0.25))
model_training.add(Dense(256))
model_training.add(Dense(128))
# model_training.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model_training.add(Dropout(0.2))
# model_training.add(Dense(128)) 
model_training.add(MaxPooling1D(pool_size=3))
# model_training.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model_training.add(Dropout(0.2))
# model_training.add(MaxPooling1D(pool_size=3))
# model_training.add(Dropout(0.2))
# model_training.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='tanh'))
# model_training.add(Dropout(0.2))
# model_training.add(MaxPooling1D(pool_size=3))
# model_training.add(Dropout(0.2))
# model_training.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='tanh'))
# model_training.add(Dropout(0.2))
# model_training.add(MaxPooling1D(pool_size=3))
# model_training.add(Dropout(0.2))
#model.add(Dropout(0.2))
model_training.add(Flatten())
model_training.add(Dense(train_labels.shape[1]))
model_training.add(Activation('sigmoid'))
model_training.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_training.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 300, 32)           288       
_________________________________________________________________
dropout (Dropout)            (None, 300, 32)           0         
_________________________________________________________________
dense (Dense)                (None, 300, 512)          16896     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 32)           65568     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 32)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 300, 256)          8448      
_________________________________________________________________
dense_2 (Dense)              (None, 300, 128)          3

In [22]:
# model = Sequential()
# model_training.add(InputLayer(input_shape=(300,1)))
# model.add(Dense(input_dim=doc2vec_dimensions,units = 90, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1200, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(400, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(600, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(units = train_labels.shape[1], activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
# model.fit(train_data, train_labels, validation_data=(test_data, test_labels), batch_size=32, epochs=15)

In [24]:
# print(train_labels[0])

In [25]:
model_training.fit(train_data, train_labels, validation_data=(test_data, test_labels), batch_size=128, epochs=50)

Epoch 1/50
61/61 [==============================] - 27s 418ms/step - loss: 0.1815 - accuracy: 0.2663 - val_loss: 0.0506 - val_accuracy: 0.3587
Epoch 2/50
61/61 [==============================] - 25s 404ms/step - loss: 0.0468 - accuracy: 0.3843 - val_loss: 0.0459 - val_accuracy: 0.5575
Epoch 3/50
61/61 [==============================] - 25s 416ms/step - loss: 0.0336 - accuracy: 0.5991 - val_loss: 0.0239 - val_accuracy: 0.7691
Epoch 4/50
61/61 [==============================] - 25s 417ms/step - loss: 0.0167 - accuracy: 0.7814 - val_loss: 0.0208 - val_accuracy: 0.7907
Epoch 5/50
61/61 [==============================] - 26s 435ms/step - loss: 0.0129 - accuracy: 0.8181 - val_loss: 0.0209 - val_accuracy: 0.7917
Epoch 6/50
61/61 [==============================] - 25s 414ms/step - loss: 0.0111 - accuracy: 0.8311 - val_loss: 0.0200 - val_accuracy: 0.7900
Epoch 7/50
61/61 [==============================] - 27s 448ms/step - loss: 0.0098 - accuracy: 0.8517 - val_loss: 0.0229 - val_accuracy: 0.7996

In [26]:
test_articles = [{'raw': reuters.raw(fileId), 'categories': reuters.categories(fileId)} for fileId in reuters.fileids() if fileId.startswith('test/')]
test_data = [d2v.infer_doc(gensim.utils.simple_preprocess(article['raw'])) for article in test_articles]

In [27]:
test_data = np.reshape(test_data, (len(test_data),300,1))

In [28]:
predictions = model_training.predict(np.asarray(test_data))

In [29]:
predictions[predictions<0.5] = 0
predictions[predictions>=0.5] = 1

In [30]:
labelBinarizer = MultiLabelBinarizer()
labelBinarizer.fit([reuters.categories(fileId) for fileId in reuters.fileids()])
predicted_labels = labelBinarizer.inverse_transform(predictions)

In [31]:
import collections
count = 0
print(len(predicted_labels))
for predicted_label, test_article in zip(predicted_labels, test_articles):
    # print('title: {}'.format(test_article['raw'].splitlines()[0]))
    # print('predicted: {} - actual: {}'.format(list(predicted_label), test_article['categories']))
    # print('')
    if collections.Counter(list(predicted_label)) == collections.Counter(test_article['categories']):
        count = count + 1
    # print(count)
print(count)

In [32]:
print(count/len(predicted_labels))

In [33]:
test_data = [d2v.infer_doc(gensim.utils.simple_preprocess(article['raw'])) for article in test_articles]
test_labels = labelBinarizer.transform([article['categories'] for article in test_articles])

In [34]:
test_data = np.reshape(test_data, (len(test_data),300,1))
test_labels = np.reshape(test_labels, (len(test_labels),90,1))

In [35]:
loss, acc = model_training.evaluate(test_data, test_labels, batch_size=128)
    
print('Score: %1.4f' % loss)
print('Accuracy: %1.4f' % acc)

24/24 [==============================] - 3s 103ms/step - loss: 0.0290 - accuracy: 0.8142
Score: 0.0290
Accuracy: 0.8142
